# Fierce Research Execution Model

##### Import Packages

In [1]:
import eikon as tr
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime

In [2]:
tr.set_app_key('AppKey')

In [3]:
trading, e= tr.get_data(['HXU.TO','NUGT.K', 'SPY', 'QQQ.O','SPXL.K','TQQQ.O'],['TR.closeprice.date','TR.closeprice','TR.PriceAvg20D','TR.PriceAvg200D'])

##### Import ETFs Prices

In [4]:
trading['Date'] = pd.to_datetime(trading['Date'])
trading.set_index('Date', inplace=True)
trading.index = trading.index.map(lambda t: t.strftime('%Y-%m-%d')) 
trading.columns = map(str.lower, trading.columns)
trading = trading.round(2)
trading

,instrument,close price,20-day sma,200-day sma
2020-06-23,HXU.TO,32.93,32.60,37.07
2020-06-23,NUGT.K,72.95,66.13,118.59
2020-06-23,SPY,312.05,310.50,301.58
2020-06-23,QQQ.O,248.84,239.25,209.46
2020-06-23,SPXL.K,43.47,42.88,50.78
2020-06-23,TQQQ.O,99.80,89.12,75.68


##### Set Tactical Asset Allocation Weights For Each ETF

In [5]:
activeportusd = 50000
trading['tactical aa'] = [.05,.05,.20,.20,.20,.30]
trading['tactical aa'].sum()

1.0

In [6]:
trading['maxnotional'] = activeportusd*trading['tactical aa']
trading['max num of shrs'] = (trading['maxnotional']/trading['close price']).round()

##### Position Size Guidelines

In [7]:
position_sizes = []
for idx, row in trading.iterrows():
    i = row['close price']
    x = row['20-day sma']
    y = row['200-day sma']
    z = row['max num of shrs']
    if (i > x) and (i > y):
        position_sizes.append(1*z)
    elif ((i > x) and (i < y)) or ((i < x) and (i > y)):
        position_sizes.append(0.5*z)
    elif (i < x) and (i < y):
        position_sizes.append(0.1*z)
    else:
        position_sizes.append(np.nan)

trading['recommended position size'] = position_sizes

##### Price close VS 20-day sma VS 200-day sma multiplied by max number of shares

In [8]:
trading

,instrument,close price,20-day sma,200-day sma,tactical aa,maxnotional,max num of shrs,recommended position size
2020-06-23,HXU.TO,32.93,32.60,37.07,0.05,2500.0,76.0,38.0
2020-06-23,NUGT.K,72.95,66.13,118.59,0.05,2500.0,34.0,17.0
2020-06-23,SPY,312.05,310.50,301.58,0.20,10000.0,32.0,32.0
2020-06-23,QQQ.O,248.84,239.25,209.46,0.20,10000.0,40.0,40.0
2020-06-23,SPXL.K,43.47,42.88,50.78,0.20,10000.0,230.0,115.0
2020-06-23,TQQQ.O,99.80,89.12,75.68,0.30,15000.0,150.0,150.0
